# Imports

In [1]:
from ortools.sat.python import cp_model
from calendar import monthrange

from scheduling_funtions import *
from itertools import permutations

# Creating Data

In [2]:
#name, ft shifts only, midnights only, first 6 months?
staff_list = [['Olivia', 0, 1, 0],
              ['Emma', 0, 1, 0], 
              ['Ava', 1, 0, 0],
              ['Charlotte', 0, 0, 0], 
              ['Sophia', 0, 0, 1],
              ['Amelia', 0, 0, 1],
              ['Isabella', 1, 0, 0],
              ['Mia', 1, 0, 0],
              ['Evelyn', 0, 0, 1],
              ['Harper', 0, 0, 0],
              ['Camila', 0, 0, 0],
              ['Gianna', 0, 0, 0],
              ['Abigail', 0, 0, 0], 
              ['Luna', 0, 0, 0],
              ['Ella', 0, 0, 0],
              ['Elizabeth', 0, 0, 0], 
              ['Sofia', 0, 0, 0],
              ['Emily', 0, 0, 0],
              ['Avery', 0, 0, 0],
              ['Mila', 0, 0, 0]]

staff = range(len(staff_list))
non_midnight_staff = list(filter(lambda x: staff_list[x][3] == 0, staff))

In [3]:
shift_list = ['0700 - 1500',
          '0730 - 1530 (FT)',
          '0930 - 1730',
          '1200 - 2000',
          '1400 - 2200',
          '1530 - 2330 (FT)',
          '1600 - 2400',
          '1800 - 0200',
          '2000 - 0400',
          '2200 - 0400',
          '2359 - 0700',
          'On call (22:00)']

shifts = range(len(shift_list))
midnight_shifts = shifts[10:11]
late_shifts = shifts[7:10]
normal_shifts = [0,2,3,4,6,7,8,9,10,11]


In [4]:
planning_period = monthrange(2021, 5)
days = range(planning_period[1])

# Creating the model

In [5]:
model = cp_model.CpModel()

In [6]:
#decision variables
#staff 'm' works shift 's' on day 'd'
x = {(m, d, s) : \
    model.NewBoolVar('s%id%is%i' % (m, d, s)) \
    for m in staff \
    for d in days \
    for s in shifts}

In [7]:
#intermediate variables
#staff 'm' works on day 'd'

#This enforces the constraint
#No two shifts same day
days_assigned = {(m, d) : \
    model.NewBoolVar('s%id%i' % (m, d)) \
    for m in staff \
    for d in days}

for m in staff:
  for d in days:
    model.Add(days_assigned[(m,d)] == sum(x[(m,d,s)] for s in shifts))

In [8]:
#intermediate variables
#staff 'm' works on day 'd' on midnight shift 's'
midnight_shifts_assigned = {(m, d) : \
    model.NewBoolVar('s%id%i' % (m, d)) \
    for m in staff \
    for d in days}

for m in staff:
  for d in days:
    model.Add(midnight_shifts_assigned[(m,d)] == sum(x[(m,d,s)] for s in midnight_shifts))

In [9]:
#intermediate variables
#staff 'm' works on day 'd' on late shift 's'
late_shifts_assigned = {(m, d) : \
    model.NewBoolVar('s%id%i' % (m, d)) \
    for m in staff \
    for d in days}

for m in staff:
  for d in days:
    model.Add(late_shifts_assigned[(m,d)] == sum(x[(m,d,s)] for s in late_shifts))

In [10]:
 obj_int_vars = []
 obj_int_coeffs = []
 obj_bool_vars = []
 obj_bool_coeffs = []

# Constraints

In [11]:
# All shifts should be taken by doctors
# No two doctors in the same shift on the same day
for d in days:
    for s in shifts:
      constraint = [x[(m,d,s)] \
                    for m in staff]
      model.Add(sum(constraint) == 1)

In [12]:
# Maximum work 7 consecutive days
# SOFT Work maximum of 5 days in a row
for m in [1]:
    variables, coeffs = add_soft_sequence_constraint(
        model=model,
        works=[days_assigned[m, d] for d in days],
        hard_min=0,
        soft_min=0,
        min_cost=0,
        soft_max=5,
        hard_max=7,
        max_cost=3,
        prefix="max_days"
    )
    obj_bool_vars.extend(variables)
    obj_bool_coeffs.extend(coeffs)

In [13]:
# # Maximum 2 midnights in a row (except for several physicians who only work midnights)
# # SOFT Max 1 midnight in a row
# for m in non_midnight_staff:
#     variables, coeffs = add_soft_sequence_constraint(
#         model=model,
#         works=[midnight_shifts_assigned[m, d] for d in days],
#         hard_min=0,
#         soft_min=0,
#         min_cost=0,
#         soft_max=1,
#         hard_max=2,
#         max_cost=3,
#         prefix="max_mshifts"
#     )
#     obj_bool_vars.extend(variables)
#     obj_bool_coeffs.extend(coeffs)


In [14]:
# # late shifts in a row maximum - late shifts are 1800, 2000, 2200. The ability to set what a late shift is would be great. LIkely better to set them as 2000 and 2200. Midnight shift should also be included.								
# for m in staff:
#     variables, coeffs = add_soft_sequence_constraint(
#         model=model,
#         works=[late_shifts_assigned[m, d] for d in days],
#         hard_min=0,
#         soft_min=0,
#         min_cost=0,
#         soft_max=3,
#         hard_max=3,
#         max_cost=3,
#         prefix="max_lshifts"
#     )
#     obj_bool_vars.extend(variables)
#     obj_bool_coeffs.extend(coeffs)

In [15]:
# # 2 days off after 3 to 7 days of work in a row
# prior = 3

# for m in staff:
#     works = list(map(lambda x: x, [days_assigned[m, d] for d in days]))
#     for length in range(1, 4):
#         for start in range(len(works) - length - 1 - prior):
#             b = [works[i + start] for i in range(prior)]
#             model.AddBoolOr(negated_bounded_span(list(map(lambda x: x.Not(), works)), start + prior, length)).OnlyEnforceIf(b)

In [16]:
# #  5 late shifts in two weeks maximum

# for m in staff:
#     for w in range(days[-1] // 7):
#         variables, coeffs = add_soft_sum_constraint(
#             model = model,
#             works = [late_shifts_assigned[m, d + w * 7] for d in range(7)],
#             hard_min=0,
#             soft_min=0,
#             min_cost=0,
#             soft_max=3,
#             hard_max=3,
#             max_cost=3,
#             prefix="max_lshifts")
#         obj_int_vars.extend(variables)
#         obj_int_coeffs.extend(coeffs)

In [17]:
# # General principle avoid shift times changing too much day to day
# # Shifts should have same start time to 2.5 hours later compared to previous shift (the 2 hours later can be relaxed to 3,4 perhaps)
# # No shifts that start more than 1.5 hours earlier than the shift on the previous day


# penalized_transitions = []

# for shift in list(permutations(shift_list[:-1], 2)):
#     t1 = float(shift[0][0:2] + '.' + shift[0][2:4])
#     t2 = float(shift[1][0:2] + '.' + shift[1][2:4])
#     if t2 - t1 > 2.5:
#         penalized_transitions.append((shift_list.index(shift[0]), shift_list.index(shift[1]), 1))
#     elif t2 - t1 < -1.5:
#         penalized_transitions.append((shift_list.index(shift[0]), shift_list.index(shift[1]), 3))

# for previous_shift, next_shift, cost in penalized_transitions:
#     for m in staff:
#         for d in days[:-1]:
#             transition = [
#                 x[m, d, previous_shift].Not(), x[m, d + 1, next_shift].Not()
#             ]
#             if cost == 0:
#                 model.AddBoolOr(transition)
#             else:
#                 trans_var = model.NewBoolVar(
#                     'transition (employee=%i, day=%i)' % (m, d))
#                 transition.append(trans_var)
#                 model.AddBoolOr(transition)
#                 obj_bool_vars.append(trans_var)
#                 obj_bool_coeffs.append(cost)

In [18]:
# # 2 days off after 3 to 7 days of work in a row
# # Defines an illegal pattern constraint
# # 11101
# for m in staff:
#   for wind in window(days, 5):
#     constraint = [days_assigned[(m,d)] \
#                   for d in wind]

#     model.AddForbiddenAssignments(constraint, [(1,1,1,0,1)])

In [19]:
# # Certain physicians work only FT shift (0730,1530 shift)
# for m in staff:
#   constraint = [x[(m,d,s)] \
#                 for d in days \
#                 for s in normal_shifts]

#   model.Add(sum(constraint) * staff_list[m][1] == 0)

In [20]:
# # No midnights for staff in their first 6 months (need way to indicate when physician is in first 6 months of practice)
# for m in staff:
#   constraint = [x[(m,d,s)] \
#                 for d in days \
#                 for s in midnight_shifts]

#   model.Add(sum(constraint) * staff_list[m][3] == 0)

In [21]:
# # 2 days off after last midnight (except on call shift).
# for m in staff:
#     variables, coeffs = add_soft_sequence_constraint(
#         model=model,
#         works=[midnight_shifts_assigned[m, d] for d in days],
#         hard_min=0,
#         soft_min=0,
#         min_cost=0,
#         soft_max=1,
#         hard_max=2,
#         max_cost=3,
#         prefix="max_mshifts"
#     )
#     obj_bool_vars.extend(variables)
#     obj_bool_coeffs.extend(coeffs)

# Solving the Model

In [22]:
model.Minimize(
        sum(obj_bool_vars[i] * obj_bool_coeffs[i]
            for i in range(len(obj_bool_vars))) +
            sum(obj_int_vars[i] * obj_int_coeffs[i]
            for i in range(len(obj_int_vars))))

solver = cp_model.CpSolver()
solver.Solve(model)

for d in days:
    print('Day', d)
    for s in shifts:
        for m in staff:
            if solver.Value(x[(m, d, s)]) == 1:
              print(staff_list[m][0], 'works shift', shift_list[s])
    print()

Day 0
Olivia works shift 0700 - 1500
Ava works shift 0730 - 1530 (FT)
Gianna works shift 0930 - 1730
Emma works shift 1200 - 2000
Harper works shift 1400 - 2200
Amelia works shift 1530 - 2330 (FT)
Charlotte works shift 1600 - 2400
Mila works shift 1800 - 0200
Elizabeth works shift 2000 - 0400
Isabella works shift 2200 - 0400
Sophia works shift 2359 - 0700
Avery works shift On call (22:00)

Day 1
Mila works shift 0700 - 1500
Emily works shift 0730 - 1530 (FT)
Charlotte works shift 0930 - 1730
Amelia works shift 1200 - 2000
Gianna works shift 1400 - 2200
Avery works shift 1530 - 2330 (FT)
Emma works shift 1600 - 2400
Camila works shift 1800 - 0200
Sophia works shift 2000 - 0400
Harper works shift 2200 - 0400
Elizabeth works shift 2359 - 0700
Olivia works shift On call (22:00)

Day 2
Olivia works shift 0700 - 1500
Sophia works shift 0730 - 1530 (FT)
Gianna works shift 0930 - 1730
Amelia works shift 1200 - 2000
Harper works shift 1400 - 2200
Emma works shift 1530 - 2330 (FT)
Sofia works sh